In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import plotly.graph_objects as go

In [35]:
# Leer los datos
file_path = 'Cleaned_Timeline_Data.csv'
df = pd.read_csv(file_path)
df = df.drop_duplicates(subset=['Swimlane', 'Phase', 'Start Date', 'End Date', 'Events'])  # evita duplicados



In [ ]:


# --- PREPROCESADO ---
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['End Date'] = pd.to_datetime(df['End Date'])
df['Point Date'] = df['End Date'].combine_first(df['Start Date']).dt.normalize()
df['Month'] = df['Point Date'].dt.to_period('M').dt.to_timestamp()
months_sorted = sorted(df['Month'].unique())
months_labels = [d.strftime('%b %Y') for d in months_sorted]

# Swimlanes
SWIMLANE_SPACING = 1.2  # Ajusta este valor para más espacio vertical
swimlanes = ['User engagement', 'Logistics', 'Data economy']
y_pos = {lane: i * SWIMLANE_SPACING for i, lane in enumerate(swimlanes)}
df['base_y'] = df['Swimlane'].map(y_pos)

# Colores pastel minimalistas
color_map = {
    'Target': '#729BFF',
    'Milestone': '#58DDAF',
    'Barrier': '#FF857F'
}
df['color'] = df['Events'].map(color_map).fillna('#A0AEC0')

# Apilar puntos con solape exacto (misma swimlane y end date normalizado)
df['group_id'] = df['Swimlane'].astype(str) + '_' + df['Point Date'].astype(str)
group_counts = df.groupby('group_id').cumcount()
df['group_offset'] = group_counts
vertical_spacing = 0.22
df['y'] = df['base_y'] + (df['group_offset'] - df.groupby('group_id')['group_id'].transform('count')/2 + 0.5) * vertical_spacing

# Eje x: todos los meses del periodo
min_month = df['Month'].min()
max_month = df['Month'].max()
all_months = pd.date_range(min_month, max_month, freq='MS')

fig = go.Figure()

# --- Líneas base de swimlane (SIEMPRE visibles) ---
for lane, y in y_pos.items():
    fig.add_trace(go.Scatter(
        x=[min_month, max_month + pd.offsets.MonthEnd(0)],
        y=[y, y],
        mode='lines',
        line=dict(color='#E6EAF0', width=1.5),
        showlegend=False,
        hoverinfo='skip',
        visible=True
    ))

# --- Líneas horizontales por evento (Reemplazo de puntos) ---
line_traces = []
for idx, row in df.iterrows():
    hover = (
        f"<b>{row['Swimlane']}</b><br>"
        f"<b>Event:</b> {row['Events']}<br>"
        f"<b>Phase:</b> {row['Phase']}<br>"
    )
    if row['Events'] == 'Target' and pd.notnull(row['Engagement Action']):
        hover += f"<b>Engagement Action:</b> {row['Engagement Action']}<br>"

    line_traces.append(go.Scatter(
        x=[row['Start Date'], row['End Date']],
        y=[row['y'], row['y']],
        mode='lines+text',
        line=dict(color=row['color'], width=12, shape='linear'),
        name=row['Events'],
        showlegend=False,
        hovertemplate=hover + "<extra></extra>",
        visible=True,
        text=[row['Row ID']],
        textposition="middle center",
        textfont=dict(size=8, color="#273043", family="Montserrat, Arial")
    ))
# Añadir todos los traces de líneas al gráfico
for trace in line_traces:
    fig.add_trace(trace)

# Leyenda ELEGANTE (SIEMPRE visible)
for event, color in color_map.items():
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=15, color=color, line=dict(width=2, color='#fff')),
        legendgroup=event,
        showlegend=True,
        name=event,
        hoverinfo="none",
        visible=True
    ))

# --- Prepara los textos dinámicos para cada mes ---
phase_texts = []
for month in months_sorted:
    df_month = df[df['Month'] == month]
    if not df_month.empty:
        phase = df_month['Phase'].iloc[0]  # Si tienes varias phases por mes, adapta esto
        min_fecha = df_month['Start Date'].min().strftime('%b %Y')
        fecha_boton = month.strftime('%b %Y')
        text = f"Phase ({min_fecha}) - ({fecha_boton})"
    else:
        text = ""
    phase_texts.append(text)

# --- BOTONES compactos y FUNCIONALES ---
N_lines = len(swimlanes)
N_points = len(line_traces)
N_legend = len(color_map)

buttons = []

# Mostrar todos los eventos (All)
visible_all = [True] * N_lines + [True] * N_points + [True] * N_legend
buttons.append(dict(
    label="All",
    method="update",
    args=[
        {"visible": visible_all},
        {"annotations": [dict(
            x=0.5, y=1.12, xref='paper', yref='paper', showarrow=False,
            text="",  # O texto por defecto
            font=dict(size=22, family="Montserrat, Arial", color="#273043"),
            xanchor='center', yanchor='top'
        )]}
    ]
))

# Un botón por mes
for i, month in enumerate(months_sorted):
    visible = [True] * N_lines
    for row in df.itertuples():
        visible.append(row.Month == month)
    visible += [True] * N_legend
    buttons.append(dict(
        label=months_labels[i],
        method="update",
        args=[
            {"visible": visible},
            {"annotations": [dict(
                x=0.5, y=1.12, xref='paper', yref='paper', showarrow=False,
                text=phase_texts[i],
                font=dict(size=22, family="Montserrat, Arial", color="#273043"),
                xanchor='center', yanchor='top'
            )]}
        ]
    ))

# --- Eje X: todos los meses, layout amplio ---
fig.update_xaxes(
    showgrid=True,
    gridcolor='#E6EAF0',
    tickmode='array',
    tickvals=all_months,
    ticktext=[d.strftime('%b %Y') for d in all_months],
    title="Date (End Date)",
    zeroline=False,
    tickfont=dict(size=13, family="Montserrat, Arial"),
    title_font=dict(size=16, family="Montserrat, Arial"),
    range=[min_month - pd.offsets.MonthBegin(1), max_month + pd.offsets.MonthEnd(0)]
)

# Eje Y limpio
fig.update_yaxes(
    tickvals=list(y_pos.values()),
    ticktext=list(y_pos.keys()),
    showgrid=False,
    zeroline=False,
    showticklabels=True,
    range=[-0.7, SWIMLANE_SPACING * (len(swimlanes)-1) + 0.7],
    ticks=""
)

# Layout profesional y espacioso con botones arriba a la derecha
fig.update_layout(
    title=None,  # SIN título ni subtítulo
    font_family="Montserrat, Arial",
    font_color="#273043",
    plot_bgcolor="#fff",
    paper_bgcolor="#fff",
    margin=dict(l=120, r=40, t=110, b=60),
    hoverlabel=dict(bgcolor="white", font_size=14, font_family="Montserrat"),
    legend=dict(
        title="Event Type",
        orientation="h",
        yanchor="bottom",
        y=1.18,
        xanchor="left",
        x=0.01,
        font=dict(size=13)
    ),
    height=650,
    updatemenus=[
        dict(
            type="buttons",
            direction="down",
            showactive=True,
            x=1.09,
            y=0.99,
            buttons=buttons,
            bgcolor="#F8FAFC",
            bordercolor="#E5E7EB",
            borderwidth=1,
            font=dict(size=14, color="#273043", family="Montserrat"),
            pad={"r": 5, "t": 5, "b": 5}
        )
    ],
    annotations=[dict(
        x=0.5, y=1.12, xref='paper', yref='paper', showarrow=False,
        text="",  # Arranca vacío
        font=dict(size=22, family="Montserrat, Arial", color="#273043"),
        xanchor='center', yanchor='top'
    )]
)

fig.show()